In [ ]:
from census import Census
from us import states
import requests
import json
import re
import pandas as pd

c = Census("f2877c84b6e99504f1db0dfe13977e1ccea6c04e", year=2021)

In [ ]:
print(states.MD.fips)
print(states.lookup('48').abbr)

In [ ]:
var_mal_age = ['B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']

var_fem_age = ['B01001_027E','B01001_028E','B01001_029E','B01001_030E','B01001_031E','B01001_032E','B01001_033E','B01001_034E','B01001_035E','B01001_036E','B01001_037E','B01001_038E','B01001_039E','B01001_040E','B01001_041E','B01001_042E','B01001_043E','B01001_044E','B01001_045E','B01001_046E','B01001_047E','B01001_048E','B01001_049E']

url = 'https://api.census.gov/data/2019/acs/acs5/variables.json'
response = requests.get(url)
data = json.loads(response.text)
variables = data['variables']

label_lst = []
label_lst_fem = []

test = variables['B01001_039E']['label']
#print(test)

for item in var_mal_age:
    label = variables[item]['label']
    new_label = re.search(r'\d.*(?=\syears)', label).group()
    if 'and over' in new_label:
        new_label = new_label.replace('and over', '+')
    elif 'to' in new_label:
        new_label = new_label.replace('to', '-')
    elif 'and' in new_label:
        new_label = new_label.replace('and', '-')
    elif re.search(r'^\d$', new_label):
        new_label = f'0 - {new_label}'
    labels = {
        item: new_label
    }

    label_lst.append(labels)

print(label_lst)

for item in var_fem_age:
    label = variables[item]['label']
    new_label = re.search(r'\d.*(?=\syears)', label).group()
    if 'and over' in new_label:
        new_label = new_label.replace('and over', '+')
    elif 'to' in new_label:
        new_label = new_label.replace('to', '-')
    elif 'and' in new_label:
        new_label = new_label.replace('and', '-')
    elif re.search(r'^\d$', new_label):
        new_label = f'0 - {new_label}'
    labels = {
        item: new_label
    }

    label_lst_fem.append(labels)

print(label_lst_fem)

In [ ]:
result_dict = {}

for variables in label_lst:
    for k, v in variables.items():
        m_data = c.acs5.get(('NAME', k), {'for': 'state:{}'.format(states.VA.fips)})
        result = m_data[0][k]
        
        age_range = v.split(' - ')
        if len(age_range) == 2:
            start_age = int(age_range[0])
            end_age = int(age_range[1])
        else:
            start_age = end_age = int(age_range[0])
        decade = f"{start_age // 10 * 10} - {end_age // 10 * 10 + 9}"
        
        result_dict[decade] = result_dict.get(decade, 0) + int(result)

data_lst = [result_dict]

print(data_lst)

In [ ]:
"""Old code
data_lst = []

for variables in label_lst:
    for k, v in variables.items():
        m_data = c.acs5.get(('NAME', k), {'for': 'state:{}'.format(states.VA.fips)})
        result = m_data[0][k]
        result_dict = {
            v: int(result)
        }
        
        data_lst.append(result_dict)

#print(data_lst)"""

In [ ]:
print(data_lst[5])

In [ ]:
print(data_lst)

In [ ]:
data_dict = {}
for d in data_lst:
    for key, value in d.items():
        data_dict.setdefault(key, []).append(value)

#print(data_dict)
nes_dict = {
    'Males': data_dict.values(),
    'Age Group': data_dict.keys()
}

#print(nes_dict)

df = pd.DataFrame(nes_dict)
df = df.sort_index(ascending=False)

print(df)

In [ ]:
def femalePop(var_fem_age_lst):

    url = 'https://api.census.gov/data/2019/acs/acs5/variables.json'
    response = requests.get(url)
    data = json.loads(response.text)
    variables = data['variables']

    label_lst = []

    for item in var_fem_age_lst:
        label = variables[item]['label']
        new_label = re.search(r'\d.*', label).group()

        labels = {
            item: new_label
        }

        label_lst.append(labels)

    #print(label_lst)

    data_lst = []

    for variables in label_lst:
        for k, v in variables.items():
            m_data = c.acs5.get(('NAME', k), {'for': 'state:{}'.format(states.VA.fips)})
            result = m_data[0][k]
            result_dict = {
                v: int(result)
            }
            data_lst.append(result_dict)

    data_dict = {}
    for d in data_lst:
        for key, value in d.items():
            data_dict.setdefault(key, []).append(value)

    nes_dict = {
    'Females': data_dict.values(),
    'Age Group': data_dict.keys()
    }
    df = pd.DataFrame(nes_dict)
    df = df.sort_values(by='Age Group', ascending=False)

    #print(df)
    return df

In [ ]:
var_fem_age = ['B01001_027E','B01001_028E','B01001_029E','B01001_030E','B01001_031E','B01001_032E','B01001_033E','B01001_034E','B01001_035E','B01001_036E','B01001_037E','B01001_038E','B01001_039E','B01001_040E','B01001_041E','B01001_042E','B01001_043E','B01001_044E','B01001_045E','B01001_046E','B01001_047E','B01001_048E','B01001_049E']

In [ ]:
df1 = df
df1 = df1.explode('Males')

df2 = femalePop(var_fem_age)
df2 = df2.explode('Females')

#result = pd.merge(df1, df2, left_index=True, right_index=True)
result = pd.merge(df1, df2, on='Age Group')
result = result.reindex(columns=['Age Group', 'Males', 'Females'])

print(result)

In [ ]:
result['Total'] = result['Males'] + result['Females']

In [ ]:
result

In [ ]:
total_pop = result['Total'].sum()
total_pop

total_state_pop = c.acs5.get(('NAME', 'B01003_001E'), {'for': 'state:{}'.format(states.VA.fips)})
#print(total_state_pop)

In [ ]:
check = total_pop - total_state_pop[0]['B01003_001E']
print(check)

# US Census Data Science

1. Compare home ownership rates across states - looks at % owner-occupied housing units with a mortgage by state
2. Analyze median household income differences across states - provides state level median household income estimates
3. Assess housing affordability by state - can look at median home values compared to median income to evaluate affordability
4. Identify most/least affordable states for housing - rank states by typical housing costs as a percentage of household income
5. Evaluate real estate tax burdens by state - median real estate taxes paid compared to median income
6. Assess mortgage debt levels - distribution of housing units with a mortgage across loan size buckets
7. Identify states with highest mortgage debt - rank by % of units with mortgages and median mortgage amount
8. Analyze trends in home values over time - compare median home values year-over-year from census data
9. Segment states by predominant housing type - cluster analysis on housing structure types by state
10. Predict home values based on housing characteristics - build a regression model to estimate home values
11. Forecast mortgage interest rates - time series analysis and forecasting of mortgage rates
12. Identify associations between demographics and home ownership - correlation analysis between ownership and factors like income, age etc.

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import re

%matplotlib inline
init_notebook_mode(connected=True) 

In [ ]:
data_2022 = pd.read_csv('../Data/AverageHouseHoldSize2022.csv', index_col=0)
data_2022= data_2022.transpose()
data_2022 = data_2022.dropna(axis=1)

data_2022.head()

In [ ]:
data_2012 = pd.read_csv('../Data/AverageHouseHoldSize2012.csv', index_col=0)
data_2012= data_2012.transpose()
data_2012 = data_2012.dropna(axis=1)

data_2012.head()

In [ ]:
state_abbreviations = {
'Alabama': 'AL',
'Alaska': 'AK',
'Arizona': 'AZ',
'Arkansas': 'AR',
'California': 'CA',
'Colorado': 'CO',
'Connecticut': 'CT',
'Delaware': 'DE',
'District of Columbia': 'DC',
'Florida': 'FL',
'Georgia': 'GA',
'Hawaii': 'HI',
'Idaho': 'ID',
'Illinois': 'IL',
'Indiana': 'IN',
'Iowa': 'IA',
'Kansas': 'KS',
'Kentucky': 'KY',
'Louisiana': 'LA',
'Maine': 'ME',
'Maryland': 'MD',
'Massachusetts': 'MA',
'Michigan': 'MI',
'Minnesota': 'MN',
'Mississippi': 'MS',
'Missouri': 'MO',
'Montana': 'MT',
'Nebraska': 'NE',
'Nevada': 'NV',
'New Hampshire': 'NH',
'New Jersey': 'NJ',
'New Mexico': 'NM',
'New York': 'NY',
'North Carolina': 'NC',
'North Dakota': 'ND',
'Ohio': 'OH',
'Oklahoma': 'OK',
'Oregon': 'OR',
'Pennsylvania': 'PA',
'Rhode Island': 'RI',
'South Carolina': 'SC',
'South Dakota': 'SD',
'Tennessee': 'TN',
'Texas': 'TX',
'Utah': 'UT',
'Vermont': 'VT',
'Virginia': 'VA',
'Washington': 'WA',
'West Virginia': 'WV',
'Wisconsin': 'WI',
'Wyoming': 'WY',
'Puerto Rico': 'PR'
}

In [ ]:
def dataframe_cleanup(df):
    
    #Clean up dataframe columns
    dict_data = df.to_dict()

    new_dict = {'State': [], 'Owner_occupied': [], 'Renter_occupied': []}
    for k, v in dict_data.items():
        if k=='Owner occupied':
            new_dict['State'].extend(list(v.keys())),
            new_dict['Owner_occupied'].extend(v.values())

        if k=='Renter occupied':
            new_dict['Renter_occupied'].extend(v.values())


    #Add state abbreviations to the dataframe column
    df = pd.DataFrame(new_dict, columns=['State', 'Owner_occupied','Renter_occupied'])
    df['Code'] = df['State'].map(state_abbreviations)
            
    return df

In [ ]:
df_2022 = dataframe_cleanup(data_2022)

In [ ]:
df_2022.head()

In [ ]:
#Make choropleth map

def make_map(df, data_year: str):

    data = dict(type = 'choropleth',
                colorscale = 'Portland',
                locations = df['Code'],
                locationmode = 'USA-states',
                z=df['Owner_occupied'],
                colorbar = {'title':'Average Number of People'})

    layout = dict(
        title = {'text': f'{data_year} US Average Household Size by Home Owner', 'x':0.5, 'xanchor': 'center'},
        geo = dict(scope = 'usa'),
        autosize = True,
        width = 800,
        height = 600,
    )

    choromap_own = go.Figure(data = [data],layout = layout)

    data_rent = dict(type = 'choropleth',
                colorscale = 'Portland',
                locations = df['Code'],
                locationmode = 'USA-states',
                z=df['Renter_occupied'],
                colorbar = {'title':'Average Number of People'})

    layout = dict(
        title = {'text': f'{data_year} US Average Household Size by Renter', 'x':0.5, 'xanchor': 'center'},
        geo = dict(scope = 'usa'),
        autosize = True,
        width = 800,
        height = 600,
    )

    choromap = go.Figure(data = [data_rent],layout = layout)

    iplot(choromap_own)
    iplot(choromap)

In [ ]:
make_map(df_2022, '2022')

In [ ]:
df_2012 = dataframe_cleanup(data_2012)

In [ ]:
df_2012.head()

In [ ]:
make_map(df_2012, '2012')

In [ ]:
def make_bar_graph(df, state: str, data_year: str):
    #grab the state for comparison
    target_state = df[df['State']==state].iloc[0]
    #conver series to dataframe
    target_state = target_state.to_frame().T

    # Melt DataFrame to have one row per category
    df_melt = target_state.melt(id_vars=['State', 'Code'], value_vars=['Owner_occupied', 'Renter_occupied'], var_name='Category', value_name='Avg Household Size')

    # Create the bar chart
    fig = px.bar(df_melt, x='Category', y='Avg Household Size', color='Category', title=f'{data_year} Comparison of Owner Occupied vs Renter Occupied for {state}')
    fig.show()


In [ ]:
make_bar_graph(df_2012, 'California', '2012')

# Population Pyramid

In [ ]:
def clean_headers(val):
    if isinstance(val, str):
        val = val.split("!!")[0]
        return val
    else:
        return val

In [ ]:
data_pop = pd.read_csv('../Data/SexByAge2022.csv', index_col=0)
data_pop = data_pop.rename(columns=clean_headers)
data_pop.head()

In [ ]:
male_data = data_pop.iloc[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]

In [ ]:
female_data = data_pop.iloc[[26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]]

In [ ]:
male_data.reset_index()

In [ ]:
female_data.reset_index()

In [ ]:
def cleanup_dataframe(df, gender: str):

    dict_data = df.to_dict()

    #Clean up label column for white spaces and convert population size to integers
    cleaned_dict = {state: {key.strip(): int(float(value.replace(',',''))) for key, value in data.items()} for state, data in dict_data.items()}

    final_dict = {'State': [], 'Age Group': [], gender: []}

    #Clean up age_group labels
    age_groups = ['0 - 4','5 - 9','10 - 14','15 - 19' ,'20 - 24','25 - 29','30 - 34', '35 - 39','40 - 44', '45 - 49','50 - 54','55 - 59','60 - 64','65 - 69','70 - 74','75 - 79', '80 - 84', '85 +']

    for state, data in cleaned_dict.items():
        gender_data = [data['Under 5 years'], data['5 to 9 years'], 
                data['10 to 14 years'],(data['15 to 17 years'] + data['18 and 19 years']),
                (data['20 years'] + data['21 years'] + data['22 to 24 years']), data['25 to 29 years'],
                data['30 to 34 years'],data['35 to 39 years'],
                data['40 to 44 years'], data['45 to 49 years'],
                data['50 to 54 years'], data['55 to 59 years'],
                (data['60 and 61 years'] + data['62 to 64 years']),(data['65 and 66 years'] + data['67 to 69 years']),
                data['70 to 74 years'],data['75 to 79 years'],
                data['80 to 84 years'],data['85 years and over']]
        
        for age_group, g in zip(age_groups, gender_data):
            final_dict['State'].append(state)
            final_dict['Age Group'].append(age_group)
            final_dict[gender].append(g)

    df = pd.DataFrame(final_dict)

    return df

In [ ]:
df_male_2022 = cleanup_dataframe(male_data,'Males')
df_female_2022 = cleanup_dataframe(male_data,'Females')


In [ ]:
df_male_2022.head()

In [ ]:
def graph_pyramid(df_male,df_female,state: str, data_year: str):

    df = pd.merge(df_male, df_female)
    #Add State Code
    df['Code'] = df['State'].map(state_abbreviations)

    target_state = df.loc[df['State']==state]
    
    y = target_state['Age Group']
    x1 = target_state['Males']
    x2 = target_state['Females'] * -1
    state = target_state['State'].iloc[0]
    state_code = target_state['Code'].iloc[0]

    #Create instance for chart figure

    fig = go.Figure()

    #Add Trace to Figure
    fig.add_trace(go.Bar(
        y = y,
        x = x1,
        name = 'Males',
        orientation='h'
    ))

    #Add Trace to figure
    fig.add_trace(go.Bar(
        y = y,
        x = x2,
        name = 'Females',
        orientation='h'
    ))

    # Find the minimum and maximum values in x1 and x2
    min_val = min(min(x1), min(x2))
    max_val = max(max(x1), max(x2))

    # Create a range of values for the x-axis ticks
    tickvals = list(range(min_val, max_val, int((max_val - min_val) / 5)))

    def human_format(num):
        magnitude = 0
        while abs(num) >= 1000:
            magnitude += 1
            num /= 1000.0
        return '%.1f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

    #Update the Title_suffix
    max_val = max(max(x1), max(x2))
    if abs(max_val) >= 10**6:
        title_suffix = 'in Millions'
    elif abs(max_val) >= 10**3:
        title_suffix = 'in Thousands'
    else:
        title_suffix = ''

    #Update Figure Layout
    fig.update_layout(
        template = 'plotly_white',
        title = f'Age Pyramid {state} {data_year}',
        title_font_size = 24,
        barmode = 'relative',
        bargap =0.0,
        bargroupgap =0,
        height = 700,
        width = 1400,
        xaxis= dict(
            tickvals=tickvals,
            ticktext=[human_format(abs(i)) for i in tickvals],
            title = 'Populations ' + title_suffix,
            title_font_size=14
        )
    )

    #Plot Figure
    fig.show()




In [ ]:
graph_pyramid(df_male_2022,df_female_2022,'California', '2022')

In [ ]:
data_pop = pd.read_csv('../Data/SexByAge2012.csv', index_col=0)
data_pop = data_pop.rename(columns=clean_headers)
data_pop.head()

In [ ]:
male_data = data_pop.iloc[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]

In [ ]:
female_data = data_pop.iloc[[26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]]

In [ ]:
male_data.reset_index()

In [ ]:
female_data.reset_index()

In [ ]:
df_male_2012 = cleanup_dataframe(male_data,'Males')
df_female_2012 = cleanup_dataframe(male_data,'Females')

In [ ]:
graph_pyramid(df_male_2012,df_female_2012,'California', '2012')

# Financial Mortgage Data

In [ ]:
def clean_mortgage_headers(val):
    if isinstance(val, str):
        if 'Percent' in val:
            val = val.split("!!")[0]
            val = val + " Percent"
        else:
            val = val.split("!!")[0]
        return val
    else:
        return val

In [ ]:
mortgage_data = pd.read_csv('../Data/Financial_Mortgate_Data_2022.csv', index_col=0)
mortgage_data = mortgage_data.rename(columns=clean_mortgage_headers)
mortgage_data.head()

In [ ]:
home_value = mortgage_data.iloc[[2,3,4,5,6,7,8,9]]

In [ ]:
def convert_value(value):
    if '%' in value:
        return float(value.replace('%', '')) / 100  # Convert percentage to a decimal
    else:
        return int(value.replace(',', ''))  # Remove commas and convert to integer

In [ ]:
def dataframe_sanitize(df):
    dict_mortgage = df.to_dict()
    cleaned_dict = {state: {key.strip(): convert_value(value) for key, value in data.items()} for state, data in dict_mortgage.items()}

    # Clean up dataframe columns
    final_list = []
    median_dict = {}  # Dictionary to store 'Median (dollars)' for each state
    for state, data in cleaned_dict.items():
        if 'Percent' not in state:
            state_percent = cleaned_dict.get(state + ' Percent', {})
            for value, unit in data.items():
                if value != 'Median (dollars)':
                    final_dict = {}
                    final_dict['State'] = state
                    final_dict['Value'] = value
                    final_dict['Units'] = unit
                    final_dict['unit_percent'] = state_percent.get(value)
                    final_list.append(final_dict)
                else:
                    median_dict[state] = unit  # Store 'Median (dollars)' in median_dict

    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)

    # Create a separate DataFrame for 'Median'
    median_df = pd.DataFrame(list(median_dict.items()), columns=['State', 'Median'])
    median_df['Code'] = median_df['State'].map(state_abbreviations)

    return df, median_df

In [ ]:
df, df_median = dataframe_sanitize(home_value)

In [ ]:
df.head()

In [ ]:
df[df['State']=='Alabama']

In [ ]:
def graph_bar(df, state: str, data_year: str):
    target_state = df[df['State']==state]
    # Create the bar chart
    fig = px.bar(target_state, x='Value', y='Units', color='Value', title=f'{data_year} Number of Housing Units by Cost in {state}')
    fig.update_xaxes(title_text='Home Value')
    fig.update_yaxes(title_text='Number of Houses')
    fig.show()


In [ ]:
graph_bar(df,'Florida','2022')

In [ ]:
def graph_bar_median_price(df, data_year: str):
    # Create the bar chart
    fig = px.bar(df, x='State', y='Median', color='State', title=f'{data_year} Median Price of Homes in US')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Median Price')
    fig.show()

In [ ]:
df_median.head()

In [ ]:
graph_bar_median_price(df_median, '2022')

# Mortgage Status

In [ ]:
mortgage_status = mortgage_data.iloc[[12,13,14,15,16]]

In [ ]:
mortgage_status

In [ ]:
df, df_median = dataframe_sanitize(mortgage_status)

In [ ]:
df.head()

In [ ]:
def graph_pie(df,state:str,data_year: str):
    target_state = df[df['State']==state]
    # Create the bar chart
    fig = px.pie(target_state, values='Units', names='Value', title=f'{data_year} Number of Houses by Mortgage Type in {state}')
    fig.show()

In [ ]:
graph_pie(df,'California','2022')

# Household Income

In [ ]:
household_income = mortgage_data.iloc[[18,19,20,21,22,23,24,25,26]]

In [ ]:
def dataframe_sanitize_income(df):
    
    dict_mortgage = df.to_dict()
    cleaned_dict = {state: {key.strip(): convert_value(value) for key, value in data.items()} for state, data in dict_mortgage.items()}

    #Clean up dataframe columns
    final_list = []
    median_dict = {}  # Dictionary to store 'Median household income (dollars)' for each state
    for state, data in cleaned_dict.items():
        if 'Percent' not in state:
            state_percent = cleaned_dict.get(state + ' Percent', {})
            for value, owners in data.items():
                if value != 'Median household income (dollars)':
                    final_dict = {}
                    final_dict['State'] = state
                    final_dict['Value'] = value
                    final_dict['Owners'] = owners
                    final_dict['Owner_percent'] = state_percent.get(value)
                    final_list.append(final_dict)
                else:
                    median_dict[state] = owners  # Store 'Median household income (dollars)' in median_dict
    
    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)

    # Create a separate DataFrame for 'Median'
    median_df = pd.DataFrame(list(median_dict.items()), columns=['State', 'Median'])
    median_df['Code'] = median_df['State'].map(state_abbreviations)
    

    return df, median_df

In [ ]:
df, df_median = dataframe_sanitize_income(household_income)
df_median = df_median.sort_values('Median')
df.head()
df_median.head()

In [ ]:
def chart_income(df, state:str,data_year:str):

    target_state = df[df['State']==state]
    fig = px.bar(target_state, x="Owners", y="Value", orientation='h', title=f'{data_year} Number of Homeowners with Household Income in {state}')
    fig.update_xaxes(title_text='Number of Homeowners')
    fig.update_yaxes(title_text='Household Income')
    fig.show()

    fig = px.pie(target_state, values='Owners', names='Value', title=f'{data_year} Percent of Homeowners by Household Income in {state}')
    fig.show()

In [ ]:
chart_income(df,'Alabama', '2022')

In [ ]:
def chart_income_median(df, data_year:str):
    fig = px.bar(df, x="State", y="Median", color='State', title=f'{data_year} Median Household Income by Homeowners in US')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Median Household Income')
    fig.show()
    

In [ ]:
chart_income_median(df_median, '2022')

# Monthly Mortgage Payments

In [ ]:
monthly_mortgage = mortgage_data.iloc[[34,35,36,37,38,39,40,41,42,43,44]]

In [ ]:
monthly_mortgage

In [ ]:
df, df_median = dataframe_sanitize(monthly_mortgage)

In [ ]:
df.head()

In [ ]:
def graph_bar_pmt(df, state: str, data_year: str):
    target_state = df[df['State']==state]
    # Create the bar chart
    fig = px.bar(target_state, x='Value', y='Units', color='Value', title=f'{data_year} Number of Homes by Monthly Mortgage Payments in {state}')
    fig.update_xaxes(title_text='Monthly Mortgage')
    fig.update_yaxes(title_text='Number of Houses')
    fig.show()

In [ ]:
graph_bar_pmt(df,'Virginia', '2022')

In [ ]:
def graph_bar_pmt_median(df, data_year: str):
    # Create the bar chart
    fig = px.bar(df, x='State', y='Median', color='State', title=f'{data_year} Median Monthly Mortgage Payments in US')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Median Monthly Payment')
    fig.show()

In [ ]:
graph_bar_pmt_median(df_median, '2022')

# Monthly Mortgage as percent of Income

In [ ]:
#monthly_mortgage_to_income = mortgage_data.iloc[[46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66]]

# Real Estate Taxes

In [ ]:
real_estate_tax = mortgage_data.iloc[[68,69,70,71,72]]

In [ ]:
real_estate_tax

In [ ]:
def dataframe_sanitize_re(df):
    
    dict_mortgage = df.to_dict()
    cleaned_dict = {state: {key.strip(): convert_value(value) for key, value in data.items()} for state, data in dict_mortgage.items()}

    #Clean up dataframe columns
    final_list = []
    median_dict = {}  # Dictionary to store 'Median (dollars)' for each state
    for state, data in cleaned_dict.items():
        if 'Percent' not in state:
            state_percent = cleaned_dict.get(state + ' Percent', {})
            for value, unit in data.items():
                if value != 'Median (dollars)':
                    final_dict = {}
                    final_dict['State'] = state
                    final_dict['Tax'] = value
                    final_dict['Units'] = unit
                    final_dict['unit_percent'] = state_percent.get(value)
                    final_list.append(final_dict)
                else:
                    median_dict[state] = unit  # Store 'Median (dollars)' in median_dict
    
    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)

    # Create a separate DataFrame for 'Median'
    median_df = pd.DataFrame(list(median_dict.items()), columns=['State', 'Median'])
    median_df['Code'] = median_df['State'].map(state_abbreviations)

    return df, median_df

In [ ]:
df, df_median = dataframe_sanitize_re(real_estate_tax)

In [ ]:
df.head()

In [ ]:
def graph_pie_tax(df,state:str,data_year: str):
    target_state = df[df['State']==state]
    # Create the bar chart
    fig = px.pie(target_state, values='Units', names='Tax', title=f'{data_year} Real Estate Taxes Paid by Household in {state}')
    fig.show()

In [ ]:
graph_pie_tax(df,'Virginia', '2022')

In [ ]:
df_median.head()

# Owner occupied Housing Units

In [ ]:
housing_units = mortgage_data.iloc[0]

In [ ]:
housing_units

In [ ]:
def clean_units_df(df):
    dict_units = df.to_dict()
    final_list = []
    for state, data in dict_units.items():
        if 'Percent' not in state:
            final_dict = {}
            final_dict['State'] = state
            final_dict['Units'] = convert_value(data)
            final_list.append(final_dict)
    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)
    
    return df

In [ ]:
df = clean_units_df(housing_units)

In [ ]:
df.head()

In [ ]:
def chart_units(df, data_year:str):

    fig = px.bar(df, x="State", y="Units", title=f'{data_year} Number of Housing Units With A Mortgage')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Number of Housing Units')
    fig.show()

In [ ]:
chart_units(df,'2022')

# Assess housing affordability by state - can look at median home values compared to median income to evaluate affordability

### Calculate house affordability: median homevalue/median income = how many years of income it would take to buy a home outright assuming no other expenses. The states with lower ratios are more affordable because income is a larger percentage of home value Remember, this is a simplified model and doesn’t take into account factors like interest rates, down payments, and other costs associated with home ownership. Also, the cost of living can vary significantly within each state, so this should be used as a general guide rather than a precise measurement of affordability.

### For more detailed analysis, you might want to consider other factors such as property taxes, insurance costs, and the percentage of income spent on housing. It’s also important to note that median values might not represent the experience of all residents, as they might be skewed by extremely high or low values.

In [ ]:
home_val, median_home =  dataframe_sanitize(home_value)

median_home.rename(columns={'Median': 'Median_House_Price'}, inplace=True)
median_home.head()

In [ ]:
hh_income, median_hh_income = dataframe_sanitize_income(household_income)
median_hh_income.rename(columns={'Median': 'Median_Income'}, inplace=True)
median_hh_income.head()

In [ ]:
house_aff = pd.merge(median_home,median_hh_income,on='State')
house_aff['Ratio'] = house_aff['Median_House_Price']/house_aff['Median_Income']
house_aff.head()

In [ ]:
def chart_income_median(df, data_year:str):
    fig = px.bar(df, x="State", y="Ratio", color='State', title=f'{data_year} Home Affordability by State')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Affordability Ratio')
    fig.show()

In [ ]:
house_aff = house_aff.sort_values('Ratio')
house_aff['Code'] = house_aff['State'].map(state_abbreviations)

In [ ]:
house_aff.head()

In [ ]:
chart_income_median(house_aff, '2022')

In [ ]:
def make_map(df, data_year: str):

    data = dict(type = 'choropleth',
                colorscale = 'Portland',
                locations = df['Code'],
                locationmode = 'USA-states',
                z=df['Ratio'],
                colorbar = {'title':'Affordability Ratio'})

    layout = dict(
        title = {'text': f'{data_year} House Affordability by State', 'x':0.5, 'xanchor': 'center'},
        geo = dict(scope = 'usa'),
        autosize = True,
        width = 800,
        height = 600,
    )

    choromap = go.Figure(data = [data],layout = layout)

    iplot(choromap)

make_map(house_aff, '2022')

In [ ]:
tax, tax_median = dataframe_sanitize_re(real_estate_tax)

In [ ]:
tax_median.rename(columns={'Median': 'Median_Tax'}, inplace=True)
tax_median.head()

In [ ]:
tax_burden = pd.merge(tax_median,median_hh_income,on='State')
tax_burden['Ratio'] = tax_burden['Median_Tax']/tax_burden['Median_Income']
tax_burden = tax_burden.sort_values('Ratio')
tax_burden['Code'] = tax_burden['State'].map(state_abbreviations)
tax_burden.head()

In [ ]:
def chart_tax_median(df, data_year:str):
    fig = px.bar(df, x="State", y="Ratio", color='State', title=f'{data_year} Real Estate Tax Burden by State')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Tax Burden Ratio')
    fig.show()

chart_tax_median(tax_burden, '2022')


# Household income accross the states

In [ ]:
data_hh_2022 = pd.read_csv('../Data/Household_income2022.csv', index_col=0)
data_hh_2022 = data_hh_2022.rename(columns=clean_headers)
data_hh_2022.head()

In [ ]:
def dataframe_clean_hh(df):
    dict_mortgage = df.to_dict()
    cleaned_dict = {state: {key.strip(): convert_value(value) for key, value in data.items()} for state, data in dict_mortgage.items()}

    # Clean up dataframe columns
    final_list = []
    total_units = {}
    for state, data in cleaned_dict.items():
        for income, unit in data.items():
            if income != 'Total:':
                final_dict = {}
                final_dict['State'] = state
                final_dict['Income'] = income
                final_dict['Units'] = unit
                final_list.append(final_dict)
            else:
                total_units[state] = unit


    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)

    df_total = pd.DataFrame(list(total_units.items()), columns=['State', 'Units'])
    df_total['Code'] = df_total['State'].map(state_abbreviations)

    return df, df_total

In [ ]:
total_hh_income, total_state_units = dataframe_clean_hh(data_hh_2022)

In [ ]:
total_hh_income.head()

In [ ]:
total_state_units = total_state_units.sort_values('Units')
total_state_units.head()

In [ ]:
def chart_income_total(df, state:str,data_year:str):

    target_state = df[df['State']==state]
    fig = px.bar(target_state, x="Units", y="Income", orientation='h', title=f'{data_year} Number of Household Units with Income in {state}')
    fig.update_xaxes(title_text='Number of Household Units')
    fig.update_yaxes(title_text='Household Income')
    fig.show()

In [ ]:
chart_income_total(total_hh_income,'Texas', '2022')

# Physical House Occupancy Characteristics

In [ ]:
def clean_house_char_headers(val):
    if isinstance(val, str):
        if 'Occupied' in val:
            val = val.split("!!")[0]
            val = val + "_total"
        elif 'Percent occupied housing units' in val:
            val = val.split("!!")[0]
            val = val + "_total_percent"
        elif 'Owner-occupied housing'in val:
            val = val.split("!!")[0]
            val = val + "_owner"
        elif 'Percent owner-occupied housing units' in val:
            val = val.split("!!")[0]
            val = val + "_own_percent"
        elif 'Renter-occupied housing units' in val:
            val = val.split("!!")[0]
            val = val + "_renter"
        elif 'Percent renter-occupied' in val:
            val = val.split("!!")[0]
            val = val + "_rent_percent"
        else:
            val = val.split("!!")[0]
        return val
    else:
        return val

In [ ]:
house_char_data = pd.read_csv('../Data/Physical_Housing_Occup.csv', index_col=0)
house_char_data = house_char_data.rename(columns=clean_house_char_headers)
house_char_data.head()

In [ ]:
units_in_struc = house_char_data.iloc[[2,3,4,5,6,7,8]]
units_in_struc.head()

In [ ]:
dict_house = units_in_struc.to_dict()
cleaned_dict = {state: {key.strip(): convert_value(value) for key, value in data.items()} for state, data in dict_house.items()}

In [ ]:
print(cleaned_dict)

In [ ]:
print(cleaned_dict.keys())

In [ ]:
new_dict = {}
final_lst = []

# Iterate over the keys and values in the original dictionary
for state_attr, attr_values in cleaned_dict.items():
    # Split the key into state and attribute
    state, attribute = state_attr.split("_", 1)
    
    # Iterate over the attribute values
    for attr, value in attr_values.items():
        # If the attribute is not in the new dictionary, add it
        if attr not in new_dict:
            new_dict[attr] = {}
        
        # If the state is not in the attribute's dictionary, add it
        if state not in new_dict[attr]:
            new_dict[attr][state] = {}
        
        # Add the value to the attribute's dictionary
        new_dict[attr][state][attribute] = value
        final_lst.append(new_dict)

In [ ]:
print(final_lst)

In [ ]:
df = pd.DataFrame(new_dict)

In [ ]:
df.head()

In [ ]:
print(final_lst[0])

In [ ]:
def dataframe_sanitize_char(df):
    dict_mortgage = df.to_dict()
    cleaned_dict = {state: {key.strip(): convert_value(value) for key, value in data.items()} for state, data in dict_mortgage.items()}

    # Clean up dataframe columns
    final_list = []
    for state, data in cleaned_dict.items():
        if 'Percent' not in state:
            state_percent = cleaned_dict.get(state + ' Percent', {})
            for value, unit in data.items():
                final_dict = {}
                final_dict['State'] = state
                final_dict['Value'] = value
                final_dict['Units'] = unit
                final_dict['unit_percent'] = state_percent.get(value)
                final_list.append(final_dict)

    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)

    return df

In [ ]:
struc_df = dataframe_sanitize_char(units_in_struc)
struc_df.head()

In [ ]:
struc_df[struc_df['State']=='Alabama']

In [ ]:
year_struc = house_char_data.iloc[[10,11,12,13,14,15,16]]
year_struc.head()

In [ ]:
rooms = house_char_data.iloc[[18,19,20,21,22]]
rooms.head()

In [ ]:
bedroom = house_char_data.iloc[[24,25,26,27]]
bedroom.head()

In [ ]:
vehicles = house_char_data.iloc[[32,33,34,35]]
vehicles.head()

In [ ]:
house_heat_fuel = house_char_data.iloc[[39,40,41,42,43,44,45]]
house_heat_fuel.tail()